<h1><center>Obtaining Stocks and Market Data + Fama-MacBeth Regression</center></h1>

## Introduction to Programming 2477-2223 S1
## Nova School of Business and Economics


### Team: Daniel Brosko (55036), ... (TODO)

TODO: theoretical description

## 1 Introduction

TODO

## 2 General packages loading

In [1]:
import pandas as pd
import numpy as np
import re
import requests
from bs4 import BeautifulSoup
import datetime
import time
import yfinance as yf
from sklearn.linear_model import LinearRegression
import statistics

#importing own functions
import scraping_functions as sf
import computing_functions as cf
import stock_prices as sp

In [2]:
# pip list # used for requirements.txt to get the version numbers

In [3]:
# loading tickers of selected stocks
tickers = sf.load_tickers("tickers_set.csv")

## 3 Obtaining the Data
### 3.1 Market Capitalization (Size) Data Scraping

Here we scrape the market capitalizations of the firms using own scraping function from scraping_function.py.
The data are in billions of USD.

In [4]:
starting_time = time.time()

# we perform the scraping of market capitalization for all desired stocks 
mkt_cap_df_list = []

for ticker in tickers:
    mkt_cap_df_list.append(sf.mkt_cap_scraper(ticker))


elapsed_time = time.time() - starting_time
print('Elapsed time:', round(elapsed_time,3) , 'seconds.')

# mkt_cap_df_list

Elapsed time: 1.807 seconds.


In [5]:
# merging mkt. capital for all stocks, and renaming the columns according to particular stocks
df_mkt_cap = mkt_cap_df_list[0]

for dataframe in mkt_cap_df_list[1:] :
    df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')

mc_columns_names = ['Date']
mc_columns_names.extend(tickers)
df_mkt_cap.columns = mc_columns_names
df_mkt_cap.set_index('Date', inplace=True)

# using size instead of mkt_cap to rule out skewness
df_size = np.log(df_mkt_cap + 0.000001)

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_83239/3308076451.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_83239/3308076451.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_83239/3308076451.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Mkt_Cap_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_mkt_cap = df_mkt_cap.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_83239/3308076451.py:5: 

### 3.2 Obtaining Stock Prices

In [6]:
# downloading stocks prices - designed to save as csv, so we have "back-up" if something goes wrong later
success_tickers, failed_tickers = sp.get_stocks(tickers)

TER - SUCCESS
ADM - SUCCESS
TMUS - SUCCESS
IDXX - SUCCESS
LNC - SUCCESS
WST - SUCCESS
AMZN - SUCCESS
DOV - SUCCESS
ORLY - SUCCESS
BSX - SUCCESS
USB - SUCCESS
ETR - SUCCESS
AZO - SUCCESS
PVH - SUCCESS
BIIB - SUCCESS
ETN - SUCCESS
PFE - SUCCESS
WMB - SUCCESS
FITB - SUCCESS
CHRW - SUCCESS
Elapsed time: 23.554 seconds.


In [7]:
# loading stock prices into list of dataframes using own function
stocks_df_list = sp.load_stocks(success_tickers)
# stocks_df_list

In [8]:
df_close_prices = stocks_df_list[0][['Date','Adj Close']]
 
for index in range(1, len(stocks_df_list)):
    dataframe = stocks_df_list[index][['Date','Adj Close']]
    df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')

cp_columns_names = ['Date']
cp_columns_names.extend(success_tickers)
df_close_prices.columns = cp_columns_names
df_close_prices['Date'] = pd.to_datetime(df_close_prices['Date']).dt.date
df_close_prices.set_index('Date', inplace=True)

/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_83239/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_83239/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn/T/ipykernel_83239/3840919972.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  df_close_prices = df_close_prices.merge(dataframe, on='Date', how = 'inner')
/var/folders/ct/m1lvzdpj1f58qmy6f7psxfwh0000gn

In [9]:
# Making returns from daily prices
# daily_returns = df_close_prices.pct_change(1) # 1 for ONE DAY lookback
daily_returns = np.log(df_close_prices/df_close_prices.shift())
daily_returns.dropna(inplace=True)
daily_returns.head()

# monthly_return = prices.pct_change(21) # 21 for ONE MONTH lookback
# annual_return = prices.pct_change(252)

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2009-12-09,0.011288,-0.010769,-0.026550,-0.005999,0.007537,-0.002321,-0.021099,-0.005363,-0.008706,0.000000,-0.003487,-0.000495,-0.003767,-0.018150,0.002286,0.000000,0.026668,0.000517,0.009766,0.000871
2009-12-10,-0.015424,0.000984,0.030908,0.009658,0.005286,0.006947,0.030525,0.001954,-0.003451,0.027909,0.005661,0.010699,-0.005839,0.007098,0.001867,0.005138,0.004377,0.013864,-0.020619,0.003995
2009-12-11,-0.003114,-0.000656,0.007220,0.002688,-0.014159,0.013245,-0.009127,0.001950,-0.001064,-0.005750,0.000868,0.015897,0.002442,0.022231,0.008049,0.001242,-0.001093,-0.001020,0.009872,0.007082
2009-12-14,0.069270,-0.008564,-0.011578,0.006878,0.012841,0.009820,-0.020865,0.012822,0.009273,0.029548,-0.000434,0.008393,0.017688,0.009099,0.004512,0.019355,0.005450,0.036586,0.014627,0.016050
2009-12-15,0.002906,0.013472,0.067536,0.008343,-0.027204,0.001502,-0.008792,-0.005786,-0.010604,-0.007870,-0.028618,-0.009115,-0.010461,-0.012068,0.011394,-0.001523,-0.005996,0.013200,-0.052670,0.011284


### 3.3 Market Return from prof. Kenneth French's website 

We dowloaded data about the market from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html , however, we need to change the date into suitable format.

In [10]:
data_market = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=3)
data_market = data_market[:-1]

data_market.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
data_market['Date'] = data_market['Date'].map(date_format_lambda)
data_market['Date'] = pd.to_datetime(data_market['Date']).dt.date
data_market.set_index('Date', inplace=True)
data_market.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009


### 3.4 Short-Time-Reversal from prof. Kenneth French's website 
We dowloaded STR data from https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
and proceed in the similar fashion to subsection 3.3.

In [11]:
st_reversal = pd.read_csv("F-F_ST_Reversal_Factor_daily.CSV", skiprows=12)
st_reversal = st_reversal[:-1]
st_reversal.rename(columns = {'Unnamed: 0': 'Date'}, inplace = True)

# using simple lambda function to put dates into correct format
date_format_lambda = lambda x: datetime.datetime.strptime(x, '%Y%m%d')
st_reversal['Date'] = st_reversal['Date'].map(date_format_lambda)
st_reversal['Date'] = pd.to_datetime(st_reversal['Date']).dt.date
st_reversal.set_index('Date', inplace=True)
st_reversal.head()

,ST_Rev
Date,
1926-01-26,0.14
1926-01-27,0.62
1926-01-28,0.22
1926-01-29,0.58
1926-01-30,0.46


### 3.4 Filtering data for reasonable time interval

In [12]:
size = df_size.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
returns = daily_returns.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
market = data_market.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")
reversal = st_reversal.query("Date >= datetime.date(2010,1,1) and Date <= datetime.date(2022,10,31)")

Here, we adjust stock returns for risk-free rate, to simplify the linear regression (we get rid of the intercept, which is rf rate).

In [13]:
# adjusting stock returns to risk-free rate
risk_free_rate = market.loc[:, 'RF']
risk_free_rate = pd.DataFrame(risk_free_rate)
risk_free_rate_column = risk_free_rate.copy()

for column in range(1, len(success_tickers)):
    risk_free_rate = pd.concat([risk_free_rate, risk_free_rate_column], axis=1)

excess_returns = returns.subtract(risk_free_rate.values)

excess_returns

,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2010-01-04,0.021209,0.005097,0.043597,0.011718,0.039407,0.006865,-0.004620,0.017864,0.009919,0.001111,0.016304,0.003781,-0.000253,0.010272,0.002613,0.010943,0.039876,0.024831,0.037247,0.010333
2010-01-05,0.003643,0.005388,0.002506,0.001663,0.020650,-0.012749,0.005883,-0.004732,0.001557,0.003324,0.025034,-0.016075,-0.008388,0.036784,-0.004859,-0.005926,-0.014366,0.012416,0.026330,-0.020946
2010-01-06,-0.009132,-0.002532,-0.069977,0.002397,0.015027,-0.009799,-0.018282,0.011790,-0.004419,0.013187,0.010600,0.005059,-0.009425,0.023640,0.000936,-0.004388,-0.003221,0.031934,0.005758,-0.013339
2010-01-07,0.005489,-0.010511,0.009352,0.037235,0.023591,0.003105,-0.017160,0.021795,-0.003131,-0.007671,0.019215,-0.012011,0.013182,0.000000,-0.008269,0.014658,-0.003770,-0.007108,0.058540,-0.009638
2010-01-08,0.010890,-0.012566,-0.057471,-0.008016,-0.015419,-0.000258,0.026717,0.028267,-0.008924,-0.009950,0.001654,-0.009638,-0.012925,-0.001604,0.020361,0.022197,0.008062,0.007993,0.000902,0.013294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-25,0.016880,0.009619,-0.001641,0.010255,0.013569,0.040840,-0.004511,-0.002004,-0.001584,-0.003222,0.010170,0.012276,0.002356,0.047404,-0.014867,0.020155,-0.009903,0.012830,0.017114,-0.007195
2022-10-26,0.021121,0.015527,-0.009508,0.005746,-0.027826,0.009612,-0.052824,-0.008881,-0.008772,0.000795,-0.001995,-0.011286,-0.017666,-0.004292,0.013803,-0.014635,-0.000743,-0.001625,-0.006484,0.006774
2022-10-27,-0.040488,-0.005247,-0.012492,-0.017047,-0.000641,-0.150705,-0.052485,-0.007322,0.033969,-0.016038,-0.018340,-0.008044,0.028674,-0.027370,-0.020171,-0.002992,-0.017972,-0.000172,-0.015516,-0.007896


Just to remind what objects (dataframes) are important fot us now, we obtained the following dataframes that we are going to use:

* size - logarithm of market capitalization    
* excess_returns - daily stock returns adjusted to risk-free rate
* market - FF market factors
* reversal - Short-Time-Reversal market factor

## 4 Intermediate Computations
### 4.1 Estimating Market Betas

We estimate stocks's sensitivity to the market (beta factors) on daily data using linear regression (implemented in the market_betas function from computation_functions.py script).

In [14]:
mkt_betas = cf.market_betas(excess_returns, market)
mkt_betas.head()

Elapsed time: 11.444 seconds.


,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2011-01-04,0.017935,0.007380,0.011974,0.009055,0.018687,0.009052,0.010824,0.013625,0.007077,0.009581,0.011858,0.006959,0.004700,0.015762,0.007382,0.012936,0.007492,0.014149,0.017356,0.008058
2011-01-05,0.017977,0.007416,0.011807,0.009080,0.018637,0.009105,0.010932,0.013653,0.007119,0.009666,0.011879,0.006986,0.004768,0.015861,0.007439,0.012991,0.007343,0.014147,0.017316,0.008082
2011-01-06,0.017931,0.007414,0.011838,0.009078,0.018613,0.009118,0.010941,0.013630,0.007117,0.009667,0.011852,0.006981,0.004751,0.015846,0.007445,0.012999,0.007362,0.014128,0.017306,0.008083
2011-01-07,0.017933,0.007398,0.011895,0.009077,0.018613,0.009125,0.010952,0.013630,0.007133,0.009669,0.011856,0.006982,0.004765,0.015847,0.007441,0.013006,0.007361,0.014116,0.017313,0.008089
2011-01-10,0.017924,0.007408,0.011905,0.009041,0.018591,0.009127,0.010977,0.013611,0.007139,0.009693,0.011842,0.006996,0.004748,0.015876,0.007452,0.012994,0.007363,0.014120,0.017248,0.008104


### 4.2 Estimating Size Betas

(TODO) We estimate stocks's sensitivity to the market (beta factors) on daily data using linear regression (implemented in the market_betas function from computation_functions.py script).

In [15]:
size_betas = cf.size_betas(excess_returns, size)
size_betas.head()

Elapsed time: 12.043 seconds.


,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2011-01-04,0.001348,0.000166,0.001597,0.000327,0.000036,0.000617,0.000139,0.000365,0.000581,-0.000579,0.000009,-0.000400,0.000603,0.000731,0.000076,0.000537,-0.000139,0.000029,0.000440,0.000300
2011-01-05,0.001227,0.000164,0.001757,0.000250,-0.000017,0.000121,0.000148,0.000324,0.000462,-0.000610,-0.000015,-0.000384,0.000531,0.000608,0.000054,0.000513,-0.000158,-0.000022,0.000376,0.000258
2011-01-06,0.001075,0.000164,0.001858,0.000256,-0.000047,0.000425,0.000156,0.000312,0.000465,-0.000605,-0.000038,-0.000371,0.000521,0.000593,0.000067,0.000533,-0.000141,-0.000039,0.000364,0.000279
2011-01-07,0.001143,0.000173,0.001728,0.000243,-0.000105,0.000432,0.000164,0.000271,0.000400,-0.000660,-0.000070,-0.000388,0.000494,0.000450,0.000080,0.000516,-0.000135,-0.000093,0.000311,0.000294
2011-01-10,0.001210,0.000177,0.001574,0.000107,-0.000148,0.000283,0.000178,0.000228,0.000405,-0.000677,-0.000097,-0.000364,0.000480,0.000319,0.000094,0.000495,-0.000125,-0.000056,0.000226,0.000308


### 4.3 Estimating STR Betas

(TODO) We estimate stocks's sensitivity to the market (beta factors) on daily data using linear regression (implemented in the market_betas function from computation_functions.py script).

In [16]:
str_betas = cf.str_betas(excess_returns, reversal)
str_betas.head()

Elapsed time: 11.343 seconds.


,TER,ADM,TMUS,IDXX,LNC,WST,AMZN,DOV,ORLY,BSX,USB,ETR,AZO,PVH,BIIB,ETN,PFE,WMB,FITB,CHRW
Date,,,,,,,,,,,,,,,,,,,,
2011-01-04,0.006474,0.003697,0.002748,0.006012,0.008347,0.004482,0.001955,0.005945,0.003569,0.003264,0.008570,0.003547,0.002324,0.005968,0.003877,0.008848,0.002873,0.005798,0.012959,0.003223
2011-01-05,0.006487,0.003700,0.002796,0.006016,0.008376,0.004481,0.001953,0.005955,0.003561,0.003260,0.008579,0.003555,0.002312,0.005966,0.003875,0.008854,0.002904,0.005811,0.012982,0.003226
2011-01-06,0.006628,0.003717,0.002669,0.006016,0.008514,0.004362,0.001920,0.005987,0.003560,0.003248,0.008767,0.003486,0.002327,0.006145,0.003816,0.008776,0.002756,0.005903,0.013103,0.003114
2011-01-07,0.006559,0.003533,0.002440,0.006058,0.008726,0.004329,0.001818,0.006149,0.003682,0.003436,0.008967,0.003566,0.002362,0.006464,0.003794,0.008825,0.002720,0.006183,0.013290,0.003030
2011-01-10,0.006578,0.003576,0.002356,0.005957,0.008675,0.004310,0.001852,0.006093,0.003688,0.003411,0.008905,0.003603,0.002343,0.006376,0.003816,0.008792,0.002749,0.006236,0.013168,0.003049


### 4.4 Transforming Data to Monthly Values

In [17]:
# firstly restricting to same time range, since to compute betas, we needed to give up of some data
size = size.iloc[253:, :]
excess_returns = excess_returns.iloc[253:, :]
market = market.iloc[253:, :]
reversal = reversal.iloc[253:, :]

In [26]:
# to monthly
monthly_size = sp.daily_to_monthly(size)
monthly_returns = sp.daily_to_monthly(excess_returns)
monthly_market = sp.daily_to_monthly(market)
monthly_reversal = sp.daily_to_monthly(reversal)
monthly_mkt_betas = sp.daily_to_monthly(mkt_betas)
monthly_size_betas = sp.daily_to_monthly(size_betas)
monthly_str_betas = sp.daily_to_monthly(str_betas)

## 5 Fama-MacBeth
### 5.1 Estimating Gamma Coefficients

In [27]:
gammas = cf.fama_macbeth(monthly_returns, monthly_mkt_betas, monthly_size_betas, monthly_str_betas)
gammas.head()

,intercept,mkt,size,str
Date,,,,
2011-01-31,-0.003352,0.092850,1.919394,0.333444
2011-02-28,-0.000650,0.265115,0.942608,-0.249411
2011-03-31,0.002725,-0.030482,1.386448,-0.368432
2011-04-30,0.005715,-0.060666,-1.655874,-0.463953
2011-05-31,0.000667,-0.178479,1.676445,-0.001223


In [28]:
results = cf.results_table(gammas)
results

,intercept,mkt,size,str
statistics,,,,
mean,-0.001332,-0.023097,0.154812,-0.00407
st. dev,0.004145,0.368386,1.547875,0.351767
t-stat,-0.321388,-0.062697,0.100016,-0.011571


In [ ]:
# TODO - regression - second step

# TODO - comment on intro/theory/findings + conclusion

# TODO - short readme update at the end

In [ ]:
# TODO - comment on findings

In [ ]:
my_date = datetime.strptime('2018-01-05', '%Y-%m-%d')
my_date.strftime('%A')